# 0. 추천시스템의 이해

## 0-1. 추천시스템의 개요

1) 정의
: 추천시스템은 사용자에게 상품을 제안하는 소프트웨어 기술이자 도구이다. 

2) 목표
: 어떤 사용자에게 어떤 상품을 *어떻게 추천*할 것인가.

3) 사례
- 당근 마켓 : 다른 사람들이 같이 본 상품을 추천
- 카카오 브런치 : 해당 글과 유사한 글을 추천
- 그 외 유튜브 등의 사례가 있다

4) 역사
2005 ~ 2010 Apriori 알고리즘 : 연관상품 추천
V
2006 ~ 2015 협업 필터링 : SVD
V
2013 ~ 2017 Spark를 이용한 빅데이터 : FP-GROW, Matrix Factorization
V
2015 ~ 2017 딥러닝을 이용한 추천시스템 : 협업 필터링 + 딥러닝, Item2Vec, Doc2Vec
V
2017 ~ 개인화 추천시스템 : Factorization Machine, Hierarchical RNN, 강화학습 + Re-Ranking, 딥러닝

## 0-2. 연관분석(Association Analysis)

1) 정의 : 룰 기반의 모델. 상품과 상품사이에 어떤 연관이 있는지 찾아낸다
- 연관의 정의 1 : 얼마나 같이 구매 되는가? (frequent)
- 연관의 정의 2 : A 아이템을 구매하는 사람이 B 아이템을 구매하는가?

장바구니 분석이라고도 한다.

2) 규칙 평과 지표
- support(지지도) : For the rule A -> B, support(A) = P(A) 조건절 A가 일어날 확률은 얼마인가?
- confidence(신뢰도) : Confidence(A -> B) = P(A,B) / P(A) A라는 상품을 구매했을 때 B라는 상품으로 이어질 확률?
- lift(향상도) : lift(A -> B) = P(A,B)/P(A)xP(B) 두 사건이 동시에 얼마나 발생하는지 비율로 독립성을 측정한다

3) 규칙 생성
- 가능한 모든 경우의 수를 탐색해서 지지도, 신뢰도, 향상도가 높은 규칙들을 찾아내는 방식

4) 문제점
- 아이템의 증가에 따른 규칙의 수가 기하급수적으로 증가

## 0-3. Apriori 알고리즘

1) 정의 : 위의 연관분석의 문제점을 해결하기 위해 아이템 셋의 증가를 줄이는 방법. 
- 빈번한 아이템셋은 하위 아이템 셋 또한 빈번할 것이다.

2) 진행 방식
- k개의 item을 가지고 단일항목집단 생성(one-item frequent set)
- 단일항목집단에서 최소 지지도 이상의 항목만 선택
- 위에서 선택한 항목만을 대상으로 2개항목집단 생성
- 2개항목집단에서 최소 지지도 이상의 항목만 선택
- 위 과정을 k개 k-item frequent set을 생성할 때까지 반복

3) 장점 : 원리가 간단하여 사용자가 쉽게 이해할 수 있고, 유의한 연관성을 갖는 구매 패턴을 찾아준다

4) 단점 : 데이터가 클 경우 속도가 느리고 연산량이 많으며, 실제 사용시 많은 연관상품이 나타나는 단점이 있다, 또한 연관상품이 상관관계는 있을 수는 있어도 인과관계는 파악하기 어렵다.

In [3]:
import mlxtend #apriori, fp-growth 사용 가능한 패키지
import numpy as np
import pandas as pd

In [5]:
#데이터셋 생성

data = np.array([
                ['우유','기저귀','쥬스'],
               ['우유','기저귀','맥주'],
               ['우유','양상추','기저귀','맥주'],
               ['양상추','맥주']
                ])

<ipython-input-5-8626bb204b0a>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array([


In [7]:
#전처리
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns = te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,False,True,False
2,True,True,True,True,False
3,False,True,True,False,False


In [8]:
%%time
from mlxtend.frequent_patterns import apriori

apriori(df, min_support = 0.5, use_colnames = True)

Wall time: 14.9 ms


,support,itemsets
0,0.75,(기저귀)
1,0.75,(맥주)
2,0.50,(양상추)
3,0.75,(우유)
4,0.50,"(맥주, 기저귀)"
5,0.75,"(기저귀, 우유)"
6,0.50,"(맥주, 양상추)"
7,0.50,"(맥주, 우유)"
8,0.50,"(맥주, 기저귀, 우유)"


## 0-4. FP-Growth 알고리즘

1) 정의 : Apriori의 속도 단점을 개선한 알고리즘으로, 아이템관의 연관성을 찾는 것은 어렵다.

2) 원리
- 모든 거래를 확인하여, 각 아이템마다의 지지도를 계산하고 최소 지지도이상의 아이템만 선택
- 모든 거래에서 빈도가 높은 아이템 순서대로 순서를 정렬
- 부모 노드를 중심으로 거래를 자식노드로 추가해주면서 tree를 생성
- 새로운 아이템이 나올 경우에는 부모노드부터 시작하고, 그렇지 않으면 기존의 노드에서 확장
- 지지도가 낮은 순서부터 시작하여, 조건부 패턴을 생성
- 모든 아이템 셋에 대해 반복
- Conditional Pattern bases를 기반으로 패턴 생성

3) 장점 : apriori 알고리즘보다 속도가 빠르고, 후보 아이템 셋을 생성할 필요없이 진행 가능

4) 단점 : 메모리를 효율적으로 사용하지 않으며, apriori 알고리즘에 비해 설계가 어렵다. 또한 지지도 계산이 FP-Tree가 만들어지고나서 가능하다.

In [9]:
#전처리
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns = te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,False,True,False
2,True,True,True,True,False
3,False,True,True,False,False


In [14]:
%%time
from mlxtend.frequent_patterns import fpgrowth, association_rules

fpgrowth(df, min_support = 0.5, use_colnames = True)

Wall time: 1.98 ms


,support,itemsets
0,0.75,(우유)
1,0.75,(기저귀)
2,0.75,(맥주)
3,0.50,(양상추)
4,0.75,"(기저귀, 우유)"
5,0.50,"(맥주, 기저귀)"
6,0.50,"(맥주, 기저귀, 우유)"
7,0.50,"(맥주, 우유)"
8,0.50,"(맥주, 양상추)"


In [15]:
association = fpgrowth(df, min_support = 0.5, use_colnames = True)
association_rules(association, metric= 'confidence', min_threshold =0.5 , support_only = False)#최소 confidence

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(기저귀),(우유),0.75,0.75,0.75,1.000000,1.333333,0.1875,inf
1,(우유),(기저귀),0.75,0.75,0.75,1.000000,1.333333,0.1875,inf
2,(맥주),(기저귀),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
3,(기저귀),(맥주),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
4,"(맥주, 기저귀)",(우유),0.50,0.75,0.50,1.000000,1.333333,0.1250,inf
5,"(맥주, 우유)",(기저귀),0.50,0.75,0.50,1.000000,1.333333,0.1250,inf
6,"(기저귀, 우유)",(맥주),0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
7,(맥주),"(기저귀, 우유)",0.75,0.75,0.50,0.666667,0.888889,-0.0625,0.75
8,(기저귀),"(맥주, 우유)",0.75,0.50,0.50,0.666667,1.333333,0.1250,1.50
9,(우유),"(맥주, 기저귀)",0.75,0.50,0.50,0.666667,1.333333,0.1250,1.50
